### <center> WEBSCRAPPING E-COMMERCE WEBSITE USING PYTHON AND SELENIUM </center>

In [1]:
#importing necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd

In [ ]:
#adding driver path (for the first time is enough)
import os
os.environ['PATH']+=r'C:\selenium_drivers'

In [ ]:
#main program - to scrape products

driver = webdriver.Chrome()

#creating a empty dataframe with columns
df = pd.DataFrame(columns=['Title', 'Name', 'Rating', 'Rating_Nos', 'Price', 'Discount', 'Distress', 'Waist Rise', 'Fade', 'Shade', 'Fit', 'Length', 'Waistband', 'Stretch'])

#first page link
main_page = "https://"#ecommerce webiste product page link"/women-jeans"

driver.implicitly_wait(2)

driver.get(main_page)

#will loop 3 pages...change the upperbound of range to scrape more pages
for i in range(0,3):
    #getting all the products in the all product containers of page
    x=driver.find_element(By.CSS_SELECTOR,'ul[class="results-base"]')
    
    #list of all the a tags of products
    products =x.find_elements(By.TAG_NAME,"a")
    
    #list of only the href (link) part of a tags
    product_links=[]
    for product in products:
        product_links.append(product.get_attribute('href'))
    
    #passing to a fuction to loop through the product links and scrape the data
    prod_loop(product_links)
    
    #getting back to the last main page
    driver.get(main_page)
    
    #reassigning the main page to return again for moving to the next page
    main_page = driver.find_element(By.CSS_SELECTOR,'a[rel="next"]').get_attribute('href')
    
    #going to next page
    driver.find_element(By.CSS_SELECTOR,'a[rel="next"]').click()
    
df.to_csv('product_details.csv',index=False)

In [3]:
#function to scrape and store data

def prod_loop(product_links):

    #loop for all the product details scraping from one single page

    for i in range(7, len(product_links)):#len(product_links)'
       
       
    #getting inside the product page
        try:
            driver.get(product_links[i])
        except e:
            print(e)
            
       
        
        #wait till the page loads
        driver.implicitly_wait(10)

        #getting the basic product details

        product_detail = driver.find_element(By.CSS_SELECTOR,'div[class="pdp-price-info"]')
        p_title = product_detail.find_element(By.CSS_SELECTOR, 'h1[class="pdp-title"]').get_attribute('innerHTML')
        p_name = product_detail.find_element(By.CSS_SELECTOR, 'h1[class="pdp-name"]').get_attribute('innerHTML')

        try:
            p_rating = product_detail.find_element(By.CSS_SELECTOR, 'div[class="index-overallRating"]').find_element(By.TAG_NAME, 'div').get_attribute('innerHTML')
            
        except:
            p_rating = np.NAN
            
        try:
            p_rating_numbers = product_detail.find_element(By.CSS_SELECTOR, 'div[class="index-ratingsCount"]').get_attribute('innerText')
        except:
            p_rating_numbers = np.NAN
            

        try:
            p_price = product_detail.find_element(By.CSS_SELECTOR,'span[class="pdp-price"]').get_attribute('innerText')
            
        except:
            p_price = np.NAN

            
        try:
            p_discount = product_detail.find_elements(By.CSS_SELECTOR,'span[class="pdp-mrp-verbiage-amt"]')[1].get_attribute('innerHTML')
           
        except:
            p_discount = np.NAN


        #creating a dictionary and updating the basic details
        details = dict()
        details.update({'Title':p_title,'Name':p_name,'Rating':p_rating,'Rating_Nos':p_rating_numbers,'Price':p_price,'Discount':p_discount})

        
        #show more button click
        try:
            driver.find_element(By.CSS_SELECTOR,'div[class="index-showMoreText"]').click()
        except:
            pass
        
        #all prod desc
        tables = driver.find_elements(By.CSS_SELECTOR, 'div[class="index-tableContainer"]')

        #prod desc table headings and values
        headings_text =[]
        values_text =[]

        for table in tables:
            headings = table.find_elements(By.CSS_SELECTOR, 'div[class="index-rowKey"]')
            for heading in headings:
                headings_text.append(heading.get_attribute('innerHTML'))
            values = table.find_elements(By.CSS_SELECTOR, 'div[class="index-rowValue"]')
            for value in values:
                values_text.append(value.get_attribute('innerHTML'))

        #updating the dictionary with desc table values
        for i in range(0,len(headings_text)):
            details.update({headings_text[i]:values_text[i]})
        
        
        
        global df
        #appending the dataframe with the dictionary values - note: for unmatched column names the values will be null
        df=df.append(details,ignore_index=True)
        